#  read data

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
from copy import deepcopy
import time
import networkx as nx
import os
import sys
import matplotlib.pyplot as plt
import pickle
import folium
from folium.plugins import HeatMap
#os.environ['PROJ_LIB'] = os.path.dirname(sys.argv[0])

In [2]:
# 55.6s
#file_path = 'data/blink_data/BLinkDataPt2_2005.csv'
file_path = 'data/blink_data/BLinkDataPt2_2005.csv'

col_names = ['c{0:02d}'.format(i) for i in range(734) ]
col_names = ['mesh_code', 'map_level','area_code', 'link_id', 'diff',
             'road_type_code','lane_num_code', 
             'start_trans_id', 'end_tras_id',
             'link_length', 'link_type_code',
             'oneway_limit_flag','stop_way_limit_flag',
             'start_lon','start_lat', 'latlng_diff_num'] + col_names
df = pd.read_csv(file_path, encoding='shift_jis', names=col_names, skiprows=3)
df.head()

,mesh_code,map_level,area_code,link_id,diff,road_type_code,lane_num_code,start_trans_id,end_tras_id,link_length,...,c724,c725,c726,c727,c728,c729,c730,c731,c732,c733
0,362257,0,13,65536,8,8,3,1096,1112,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,362257,0,13,65545,16,8,3,1112,1114,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,362257,0,13,65562,64,8,3,1114,1116,352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,362257,0,13,65627,73,8,3,1116,9766118,425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,362257,0,13,65701,31,8,3,9766118,1004,174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12789536 entries, 0 to 12789535
Columns: 750 entries, mesh_code to c733
dtypes: float64(732), int64(18)
memory usage: 71.5 GB


In [4]:
# restrict df into 首都高部分
new_df=df[df['start_lon'].between(139.537*128*3600, 139.947*128*3600, inclusive=True) & df['start_lat'].between(35.36*128*3600, 35.90*128*3600, inclusive=True)]

/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """Entry point for launching an IPython kernel.


In [5]:
new_df['combine_diff'] = [[e for e in row if e==e] for row in new_df[new_df.columns[16:]].values.tolist()]

/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
new_df=new_df[['mesh_code', 'map_level','area_code', 'link_id', 'diff',
             'road_type_code','lane_num_code', 
             'start_trans_id', 'end_tras_id',
             'link_length', 'link_type_code',
             'oneway_limit_flag','stop_way_limit_flag',
             'start_lon','start_lat', 'latlng_diff_num', "combine_diff"]]

In [7]:
new_df

,mesh_code,map_level,area_code,link_id,diff,road_type_code,lane_num_code,start_trans_id,end_tras_id,link_length,link_type_code,oneway_limit_flag,stop_way_limit_flag,start_lon,start_lat,latlng_diff_num,combine_diff
6826372,533904,0,27,5647656,16,8,2,5020486,5021661,86,1,0,0,64306431,16293947,4,"[-98.0, 154.0, -39.0, 56.0, -24.0, 64.0, -14.0..."
6826373,533904,0,27,5647673,8,8,2,5021661,5021438,23,1,0,0,64306256,16294272,2,"[-4.0, 26.0, -32.0, 61.0]"
6826374,533904,0,27,5647682,16,8,2,5021438,5021439,52,1,0,0,64306220,16294359,1,"[-182.0, 155.0]"
6826375,533904,0,27,5647699,8,8,2,5021439,5020487,22,1,0,0,64306038,16294514,2,"[-32.0, 39.0, -35.0, 33.0]"
6826376,533904,0,27,5647708,8,8,2,5020487,5020489,38,1,0,0,64305971,16294586,2,"[-21.0, 75.0, -14.0, 82.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7873403,533967,0,27,119345661,8,8,3,9906705,6242043,23,1,0,0,64483781,16529645,1,"[-18.0, -96.0]"
7873404,533967,0,27,119345679,24,8,3,9873991,6221946,126,1,0,0,64484667,16529917,3,"[-53.0, -2.0, -288.0, 39.0, -292.0, 40.0]"
7873410,533967,0,27,119485056,16,8198,2,9917801,6207004,80,1,0,0,64478023,16516130,3,"[32.0, -16.0, 21.0, -19.0, 46.0, -277.0]"
7873411,533967,0,27,119485082,8,8198,3,6211218,9917802,45,1,0,0,64478027,16516383,3,"[17.0, -103.0, 28.0, -23.0, 46.0, -33.0]"


In [ ]:
## 前点に基づいた加算
single_line= new_df.iloc[0]
display(single_line)
print(single_line["combine_diff"],len(single_line["combine_diff"]))

new_combine_list=[]
for i in range(len(single_line["combine_diff"])):
    if i== 0:
        new_combine_list.append(single_line["start_lon"]+single_line["combine_diff"][i])
    elif i==1:
        new_combine_list.append(single_line["start_lat"]+single_line["combine_diff"][i])
    else:
        new_combine_list.append(new_combine_list[i-2]+single_line["combine_diff"][i])


point_list= [[]]*(int(len(single_line["combine_diff"])/2)+1)
for i in range(len(point_list)):
    if i ==0:
        point_list[i]=[single_line["start_lat"]/128/3600,single_line["start_lon"]/128/3600] #/128/3600
    else:
        point_list[i]=[new_combine_list[i*2-1]/128/3600, new_combine_list[i*2-2]/128/3600]

#確認用 必要なければコメントアウト
# maps=folium.Map(location= point_list[0], zoom_start=15,title="Stamen Terrain")
# folium.PolyLine(locations=point_list,color="red").add_to(maps)
# degree = vincenty_inverse(point_list[-2], point_list[-1])['azimuth1']
# folium.RegularPolygonMarker(location=point_list[-1], color="red", number_of_sides=3, radius=10, rotation=270 + degree,fill=True,fill_opacity=1).add_to(maps)
# display(maps)

In [ ]:
## get the last point
end_lon_list, end_lat_list =[],[]
for i in tqdm(range(len(new_df))):
    single_line = new_df.iloc[i]
    new_combine_list=[]
    for m in range(len(single_line["combine_diff"])):
        if m== 0:
            new_combine_list.append(single_line["start_lon"]+single_line["combine_diff"][m])
        elif m==1:
            new_combine_list.append(single_line["start_lat"]+single_line["combine_diff"][m])
        else:
            new_combine_list.append(new_combine_list[m-2]+single_line["combine_diff"][m])
            
    point_list= [[]]*(int(len(single_line["combine_diff"])/2)+1)
    for n in range(len(point_list)):
        if n ==0:
            point_list[n]=[single_line["start_lat"],single_line["start_lon"]] #/128/3600
        else:
            point_list[n]=[new_combine_list[n*2-1], new_combine_list[n*2-2]]
    end_lon = point_list[-1][1]
    end_lat = point_list[-1][0]
    #print(start_lon,start_lat,combine_diff,end_lon,end_lat)
    end_lon_list.append(end_lon)
    end_lat_list.append(end_lat)

print(len(end_lon_list),len(end_lat_list))


In [ ]:
print(end_lat_list[0]/128/3600,end_lon_list[0]/128/3600)
new_df["end_lat"]=end_lat_list
new_df["end_lon"]=end_lon_list
new_df

In [ ]:
print('Unique area codes: ', new_df.area_code.unique())
print('No. of unique road types: ', new_df.road_type_code.nunique())
print('No. of unique link_ids: ', new_df.link_id.nunique())
print('No. of duplicated link_ids:', new_df.link_id.duplicated().sum())

In [ ]:
blink = pd.read_csv('data/blink_data/BlinkData.csv', encoding='shift_jis')
blink = blink[blink.roadname.notnull()]
blink.columns = ['area_code', 'link_id', 'diff', 'roadname', 'routenumber',
       'roadtype', 'link_type_code', 'tollroad', 'motorway', 'hwy', 'oneway_limit_flag',
       'speedlimit', 'hwyupdown']
blink

In [ ]:
print('Unique Area Codes:', blink.area_code.unique())
print('No. of unique roadnames:', blink.roadname.nunique())
print('No. of unique route numbers:', blink.routenumber.nunique())
print('No. of unique link_ids:', blink.link_id.nunique())
print('No. of duplicated link_ids:', blink.link_id.duplicated().sum())
print('No. of blink link_id not present in the main df:', len(blink) - blink.link_id.isin(df.link_id).sum())

In [ ]:
#Merging of both dataframes - df and blink (on common columns)
blinkmerged = pd.merge(blink, new_df[['mesh_code', 'area_code', 'link_id', 'diff', 'road_type_code', 'lane_num_code', 
                                  'start_trans_id', 'end_tras_id', 'link_length', 'link_type_code', 'start_lon', 'start_lat',"end_lon","end_lat"]], 
                       how='left', on=['area_code', 'link_id', 'diff'])

In [ ]:
blinkmerged.isna().sum()

In [ ]:
for road in list(blinkmerged["roadname"].unique()):
    if "北西" in road:
        print(road)

In [ ]:
# len=25
capital_road_list =[
    "首都高速湾岸線",
    "首都高速神奈川３号狩場線",
    "首都高速神奈川２号三ツ沢線",
    "首都高速神奈川１号横羽線",
    "首都高速１号羽田線",
    "首都高速神奈川５号大黒線",
    "首都高速神奈川６号川崎線",
    "首都高速３号渋谷線",
    "首都高速２号目黒線",
    "首都高速４号新宿線",
    "首都高速中央環状線",
    "首都高速都心環状線",
    "首都高速１号上野線",
    "首都高速１１号台場線",
    "首都高速１０号晴海線",
    "首都高速９号深川線",
    "首都高速５号池袋線",
    "首都高速埼玉大宮線",
    "首都高速八重洲線",
    "首都高速６号向島線",
    "首都高速７号小松川線",
    "首都高速６号三郷線",
    "首都高速川口線",
    "首都高速埼玉新都心線",
    "首都高速神奈川７号横浜北線"
]

In [ ]:
blinkmerged_capital=blinkmerged.loc[blinkmerged["roadname"].isin(capital_road_list)]
blinkmerged_capital

In [ ]:
### restrict the lat lon to  and mmlatitude <= 35.90 and mmlatitude >= 35.36  and mmlongitude <= 139.947   and mmlongitude >= 139.537
blinkmerged_capital_new= blinkmerged_capital[blinkmerged_capital['start_lon'].between(139.537*128*3600, 139.947*128*3600, inclusive=True)& blinkmerged_capital['start_lat'].between(35.36*128*3600, 35.90*128*3600, inclusive=True)]

In [ ]:
blinkmerged_capital_new

In [ ]:
blinkmerged_capital_new.info()

# analysis data for longest path
## step0： create connect_dic  {linkid:[linkid1,linkid2]}

In [ ]:
connect_dic={}
for linkid in tqdm(blinkmerged_capital_new["link_id"]):
    for _end_crs_id in blinkmerged_capital_new[blinkmerged_capital_new['link_id']==linkid]['end_tras_id'].values:
        for end_link in blinkmerged_capital_new[blinkmerged_capital_new['start_trans_id']==_end_crs_id]['link_id'].values:
            if linkid not in connect_dic.keys():
                connect_dic[linkid]=[end_link]
            else:
                if end_link not in connect_dic[linkid]:
                    connect_dic[linkid].append(end_link)

In [ ]:
len(connect_dic.keys())

In [ ]:
connect_dic1=deepcopy(connect_dic)
G = nx.DiGraph()
for key in connect_dic1.keys():
    for value in connect_dic1[key]:
        G.add_edge(key, value)

In [ ]:
print(G.number_of_nodes())

In [ ]:
nodes = list(G.nodes)

## create adjacency_matrix

In [ ]:
#M1 = nx.adjacency_matrix(G,nodelist=nodes)
M1=nx.to_pandas_adjacency(G, nodelist=nodes, dtype=int)

In [ ]:
len(M1)

In [ ]:
M1

In [ ]:
type(M1)

In [ ]:
#np.savetxt('result/visualization/matrix.txt',M1.toarray(), fmt='%d',)
M1.to_csv('result/metrix/capital_01_relation.csv')

## get all path

In [ ]:
roots = []
leaves = []
for node in G.nodes :
    if G.in_degree(node) == 0 : # it's a root
        roots.append(node)
    elif G.out_degree(node) == 0 : # it's a leaf
        leaves.append(node)
path_list=[]       
for root in tqdm(roots) :
    for leaf in tqdm(leaves) :
#         longest_num=0
#         longest_path=[]
#         for path in tqdm(nx.all_simple_paths(G, root, leaf)) :
#             if len(path) > longest_num:
#                 longest_num=len(path)
#                 longest_path=path
        longest_path=max(nx.all_simple_paths(G, root, leaf, cutoff=100), key=lambda x: len(x),default=1)
        if longest_path!=1:
            path_list.append(longest_path)
        print(len(path_list))


In [ ]:
print(len(path_list[1])

In [ ]:
with open('result/metrix/capital_longest_path.pkl', 'wb') as fp:
    pickle.dump(path_list, fp)

##  create distance adjacency_matrix

In [ ]:
# distance M2
from haversine import haversine, Unit

G2 = nx.DiGraph()
for key in tqdm(connect_dic1.keys()):
    for value in connect_dic1[key]:
        key_latlon=(df[df['link_id']==key]['start_lat'].values[0]/128/3600,df[df['link_id']==key]['start_lon'].values[0]/128/3600)
        value_latlon=(df[df['link_id']==value]['start_lat'].values[0]/128/3600,df[df['link_id']==value]['start_lon'].values[0]/128/3600)
        G2.add_edge(key, value, weight=haversine(key_latlon, value_latlon)*1000)

#df[df['link_id']==83585420]['start_lat'].values[0]/128/3600

In [ ]:
M2=nx.to_pandas_adjacency(G2, nodelist=nodes, dtype=float)

In [ ]:
M2

In [ ]:
for row in tqdm(M2.index):
    for column in M2:
        key_latlon=(df[df['link_id']==row]['start_lat'].values[0]/128/3600,  df[df['link_id']==row]['start_lon'].values[0]/128/3600)
        value_latlon=(df[df['link_id']==column]['start_lat'].values[0]/128/3600,df[df['link_id']==column]['start_lon'].values[0]/128/3600)
        M2.at[row,column]=haversine(key_latlon, value_latlon)*1000

In [ ]:
M2

In [ ]:
M2.to_csv('result/metrix/capital_distance_relation.csv')

## create capital_graph_link_info.csv

In [ ]:
blinkmerged_capital_new_graph= blinkmerged_capital_new.loc[blinkmerged_capital_new["link_id"].isin(nodes)][["link_id","roadname","start_lon","start_lat","end_lon","end_lat"]]

In [ ]:
blinkmerged_capital_new_graph=blinkmerged_capital_new_graph.reset_index(drop=True)

In [ ]:
blinkmerged_capital_new_graph=blinkmerged_capital_new_graph.apply({"link_id": lambda x:x,"roadname": lambda x:x, "start_lon": lambda x:x/128/3600, "start_lat": lambda x:x/128/3600, "end_lon": lambda x:x/128/3600, "end_lat": lambda x:x/128/3600})

In [ ]:
print(blinkmerged_capital_new_graph["roadname"].unique())
print(len(blinkmerged_capital_new_graph["roadname"].unique()))

In [ ]:
blinkmerged_capital_new_graph

In [ ]:
blinkmerged_capital_new_graph.to_csv('result/metrix/capital_graph_link_info_add_end_latlon.csv')